In [51]:
# importing necesarry libraries

import collections
import folium
import json 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import string
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder


In [52]:
# Initialize an empty list to store the data
json_data = []

# Define the number of lines to read (54380)
lines_to_read = 54380

# Open the JSON file and read the specified number of lines
with open(r"C:\Users\nyaki\Downloads\yelp_academic_dataset_business.json", 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):  
        if i >= lines_to_read:       
            break
        # Load each line as a JSON object
        json_object = json.loads(line)
        json_data.append(json_object)

# Convert the json_data list into a pandas DataFrame
business_df = pd.DataFrame(json_data)

# Display the first few rows of the DataFrame
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [53]:
import csv

chunk_size = 1000  # adjust this value according to your memory constraints

with open(r"C:\Users\nyaki\Downloads\yelp_academic_dataset_review.json", 'r') as f:
    with open('yelp_review_sample.csv', 'w', newline='') as csvfile:
        fieldnames = ['review_id', 'user_id', 'business_id', 'stars', 'date', 'text', 'useful', 'funny', 'cool']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        count = 0
        for line in f:
            data = json.loads(line)  # parse each line as a separate JSON object
            writer.writerow(data)
            count += 1
            if count >= 54380:
                break
 

In [54]:
review_df = pd.read_csv('yelp_review_sample.csv')

# Display the first few rows of the DataFrame
review_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,2018-07-07 22:09:11,"If you decide to eat here, just be aware it is...",0,0,0
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,2012-01-03 15:28:18,I've taken a lot of spin classes over the year...,1,0,1
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,2014-02-05 20:30:30,Family diner. Had the buffet. Eclectic assortm...,0,0,0
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,2015-01-04 00:01:03,"Wow! Yummy, different, delicious. Our favo...",1,0,1
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,2017-01-14 20:54:15,Cute interior and owner (?) gave us tour of up...,1,0,1


In [55]:
# viewing the datasets features/columns
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54380 entries, 0 to 54379
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   54380 non-null  object 
 1   name          54380 non-null  object 
 2   address       54380 non-null  object 
 3   city          54380 non-null  object 
 4   state         54380 non-null  object 
 5   postal_code   54380 non-null  object 
 6   latitude      54380 non-null  float64
 7   longitude     54380 non-null  float64
 8   stars         54380 non-null  float64
 9   review_count  54380 non-null  int64  
 10  is_open       54380 non-null  int64  
 11  attributes    49543 non-null  object 
 12  categories    54342 non-null  object 
 13  hours         45975 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 5.8+ MB


In [56]:

review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54380 entries, 0 to 54379
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   review_id    54380 non-null  object 
 1   user_id      54380 non-null  object 
 2   business_id  54380 non-null  object 
 3   stars        54380 non-null  float64
 4   date         54380 non-null  object 
 5   text         54380 non-null  object 
 6   useful       54380 non-null  int64  
 7   funny        54380 non-null  int64  
 8   cool         54380 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.7+ MB


In [57]:
# Merge the two datasets
# merging the two datasets into one using the business_id primary key

restaurant_df=pd.merge(left=review_df , right=business_df, how='left', on='business_id')

# previewing the new merge dataset
restaurant_df.head()

,review_id,user_id,business_id,stars_x,date,text,useful,funny,cool,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,2018-07-07 22:09:11,"If you decide to eat here, just be aware it is...",0,0,0,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,2012-01-03 15:28:18,I've taken a lot of spin classes over the year...,1,0,1,Body Cycle Spinning Studio,...,PA,19119,39.952103,-75.172753,5.0,144,0,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Active Life, Cycling Classes, Trainers, Gyms, ...","{'Monday': '6:30-20:30', 'Tuesday': '6:30-20:3..."
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,2014-02-05 20:30:30,Family diner. Had the buffet. Eclectic assortm...,0,0,0,Kettle Restaurant,...,AZ,85713,32.207233,-110.980864,3.5,47,1,"{'RestaurantsReservations': 'True', 'BusinessP...","Restaurants, Breakfast & Brunch",None
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,2015-01-04 00:01:03,"Wow! Yummy, different, delicious. Our favo...",1,0,1,Zaika,...,PA,19114,40.079848,-75.025080,4.0,181,1,"{'Caters': 'True', 'Ambience': '{'romantic': F...","Halal, Pakistani, Restaurants, Indian","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21..."
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,2017-01-14 20:54:15,Cute interior and owner (?) gave us tour of up...,1,0,1,Melt,...,LA,70119,29.962102,-90.087958,4.0,32,0,"{'BusinessParking': '{'garage': False, 'street...","Sandwiches, Beer, Wine & Spirits, Bars, Food, ...","{'Monday': '0:0-0:0', 'Friday': '11:0-17:0', '..."


In [58]:
restaurant_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54380 entries, 0 to 54379
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   review_id     54380 non-null  object 
 1   user_id       54380 non-null  object 
 2   business_id   54380 non-null  object 
 3   stars_x       54380 non-null  float64
 4   date          54380 non-null  object 
 5   text          54380 non-null  object 
 6   useful        54380 non-null  int64  
 7   funny         54380 non-null  int64  
 8   cool          54380 non-null  int64  
 9   name          54380 non-null  object 
 10  address       54380 non-null  object 
 11  city          54380 non-null  object 
 12  state         54380 non-null  object 
 13  postal_code   54380 non-null  object 
 14  latitude      54380 non-null  float64
 15  longitude     54380 non-null  float64
 16  stars_y       54380 non-null  float64
 17  review_count  54380 non-null  int64  
 18  is_open       54380 non-nu

DATA PREPARATION

## Data Preparation

In this section, we will perform data cleaning to prepare the dataset for analysis, the various data cleaning methods that are to be used will be;

- Renaming columns
- Checking Dealing with missing data
- Checking and removing duplicates 
- Feature Engineering
- Selecting the Relevant Columns
- Dropping Irrelevant columns
- Selecting relevant rows

In [59]:
#Renaming the columns stars_x and star_y into rarings and b/s rating
restaurant_df.rename(columns={'stars_x':'rating', 'stars_y':'b/s_rating'}, inplace=True)

Dealing with missing data

In [60]:
# checking for columns with missing values
restaurant_df.isna().sum()

review_id          0
user_id            0
business_id        0
rating             0
date               0
text               0
useful             0
funny              0
cool               0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
b/s_rating         0
review_count       0
is_open            0
attributes       842
categories         2
hours           3101
dtype: int64

In [61]:

# Filling missing values in the attributes column with "Not-Available" 
restaurant_df.attributes.fillna(value="Not-Available", inplace=True)
# Filling  missing values in the categories column with "Not-Available" 
restaurant_df.categories.fillna(value="Not-Available", inplace=True)
# Filling  missing values in the hours column with "Not-Available" 
restaurant_df.hours.fillna(value="Not-Available", inplace=True)


In [62]:
# checking for columns with missing values
restaurant_df.isna().sum()

review_id       0
user_id         0
business_id     0
rating          0
date            0
text            0
useful          0
funny           0
cool            0
name            0
address         0
city            0
state           0
postal_code     0
latitude        0
longitude       0
b/s_rating      0
review_count    0
is_open         0
attributes      0
categories      0
hours           0
dtype: int64

In [63]:
restaurant_df.dtypes

review_id        object
user_id          object
business_id      object
rating          float64
date             object
text             object
useful            int64
funny             int64
cool              int64
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
b/s_rating      float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

Dropping unnecesarry columns

The columns **review_id , useful , funny , cool , is_open , postal_code and date** are not informative in our analysis and will not be used during analysis, therefore we drop them.

In [64]:
# droping unnecessarry columns
cols=['review_id', 'useful','postal_code','funny', 'cool', 'is_open', 'date']
restaurant_df.drop(columns=cols, axis=1, inplace=True)

restaurant_df.columns

Index(['user_id', 'business_id', 'rating', 'text', 'name', 'address', 'city',
       'state', 'latitude', 'longitude', 'b/s_rating', 'review_count',
       'attributes', 'categories', 'hours'],
      dtype='object')

Dealing with duplicates

In [65]:
restaurant_df['attributes']



0        {'NoiseLevel': 'u'average'', 'HasTV': 'False',...
1        {'BusinessAcceptsCreditCards': 'True', 'GoodFo...
2        {'RestaurantsReservations': 'True', 'BusinessP...
3        {'Caters': 'True', 'Ambience': '{'romantic': F...
4        {'BusinessParking': '{'garage': False, 'street...
                               ...                        
54375    {'RestaurantsAttire': 'u'casual'', 'BusinessAc...
54376    {'HasTV': 'True', 'WiFi': ''no'', 'Restaurants...
54377    {'BusinessAcceptsCreditCards': 'True', 'BikePa...
54378    {'RestaurantsTakeOut': 'True', 'RestaurantsPri...
54379    {'RestaurantsTableService': 'True', 'BusinessA...
Name: attributes, Length: 54380, dtype: object

FEATURE ENGINEERING

In feature engineering, we will combine the **city**, **state**, **address** columns to form one column, **location** column with the intention of performing geospatial analysis or visualization. Having a unified location column simplifies the process and you can use the location column directly to create maps or explore geographic trends.

In [67]:
# combining the address columns
restaurant_df['location']=restaurant_df[['city','state','address']]\
            .apply( lambda x: f"State:{x['state']}, City:{x['city']}, Address:{x['address']} ", axis=1)

# then we drop the combined columns
restaurant_df.drop(columns=['state', 'city','address'], axis=1, inplace=True)

restaurant_df.location

0        State:PA, City:North Wales, Address:1460 Bethl...
1        State:PA, City:Philadelphia, Address:1923 Ches...
2        State:AZ, City:Tucson, Address:748 W Starr Pas...
3        State:PA, City:Philadelphia, Address:2481 Gran...
4        State:LA, City:New Orleans, Address:2549 Banks...
                               ...                        
54375       State:ID, City:Boise, Address:590 E Boise Ave 
54376    State:MO, City:Saint Louis, Address:1 S Broadway 
54377    State:CA, City:Santa Barbara, Address:119 E Co...
54378    State:LA, City:New Orleans, Address:1418 Magaz...
54379    State:PA, City:West Chester, Address:1233 W Ch...
Name: location, Length: 54380, dtype: object